In [1]:
import os
import time
import json
import datetime as dt
import numpy as np
import pandas as pd
from scipy import stats
import gc

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn import metrics
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV, StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.externals.joblib import parallel_backend

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from bayes_opt import BayesianOptimization

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = lambda x : "{:,.2f}".format(x)
plt.rcParams['figure.figsize'] = (12,8)

## Agenda:
    - Load in train data set with selected engineered features
    - Train a number of different models and evaluate
        - Where possible use grid search
        - For larger parameter spaces use Bayesian Optimization or sequential parameter tuning.
    - Load in Test dataset and transform features so they align with our training dataset
    - Make predictions with best models
    - Ensemble model predictions

### Load In Dataset

In [3]:
# raw train and test
train = pd.read_csv('clean_data/train.csv', dtype={'Id':str})
test = pd.read_csv('clean_data/test.csv', dtype={'Id':str})

all_test_df = pd.read_csv('clean_data/all_test_df.csv')

train.shape, test.shape, all_test_df.shape

((15120, 54), (565892, 55), (565892, 115))

In [4]:
# w engineered features
poly_train = pd.read_csv('clean_data/train_poly_final.csv')
poly_train.shape

(15120, 200)

In [5]:
# Reduce in-memory size of pandas dataframe by compressing dtypes
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtypes

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        #else: df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [6]:
poly_train = reduce_mem_usage(poly_train)
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)
all_test_df = reduce_mem_usage(all_test_df)

ytrain = train['Cover_Type']

Memory usage of dataframe is 23.07 MB
Memory usage after optimization is: 5.77 MB
Decreased by 75.0%
Memory usage of dataframe is 6.23 MB
Memory usage after optimization is: 1.01 MB
Decreased by 83.8%
Memory usage of dataframe is 237.46 MB
Memory usage after optimization is: 38.32 MB
Decreased by 83.9%
Memory usage of dataframe is 496.50 MB
Memory usage after optimization is: 104.70 MB
Decreased by 78.9%


### Modeling:

    - Logistic Regression
    - LDA
    - KNN
    - SVM
    - Random Forest and ExtraTrees
    - Multi Layer Perceptron
    - XGBoost
    - LightGBM

In [7]:
# no nulls
poly_train.isnull().sum().sum()

0

In [8]:
# Config
seed=1111
NCV=4 

def grid_search(mod, x, y, param_grid, scale=False):
    """
    Set up and perform grid-search on data with passed algorithm using Stratified K Fold CV
    """
    cv = StratifiedKFold(NCV, shuffle=True, random_state=seed)
    grid = GridSearchCV(mod, param_grid, scoring='accuracy', n_jobs=-1, verbose=1,
                        cv=cv)
    
    # optionally scale data where required (eg nearest neighbors/svm)
    if scale:
        x = scale.fit_transform(x)
        
    # not sure what happened but multi-processing was freezing...
    with parallel_backend('threading'):
        grid.fit(x,y)
    
    print("Best Score:", grid.best_score_)
    print(grid.best_params_)
    return grid

def bayes_search(mod, x, y, param_bounds, default_params=None, ninit=10, niter=50):
    """
    For models where the parameter space is too large, turn to bayesian optimization
    to approximate best parameters.
    
    The outer function sets up and runs the bayesian optimization. 
    The inner function is our objective function to maximize.
    
    https://github.com/fmfn/BayesianOptimization
    """
    default_params = default_params or {}
    
    # infer type of parameters and cast categorical params to ints
    param_dtypes = dict.fromkeys(list(param_bounds.keys()))
    cat_params_map = {} # for exploring categorical/str parameter options
    for p, val in param_bounds.items():
        # check all same type
        assert all([type(inner) == type(val[0]) for inner in val])
        
        param_dtypes[p] = type(val[0])
        if not isinstance(val[0], (int, float)):
            print(val)
            # for str variables, options should be a list/tuple of possible values
            cat_params_map[p] = {i:v for i,v in enumerate(val)} 
            param_bounds[p] = (0, len(val)-1)
            
    print(param_bounds)   
    
    # Inner function will take numeric params and change them to appropriate dtypes
    # Then initiate and cross_val the estimator
    def mod_func(**params):
        for p,v in params.items():
            if param_dtypes[p] == int:
                params[p] = int(np.round(v,0))
            elif param_dtypes[p] != float: # all non numeric are categorical (includes tuples for mlp)
                params[p] = cat_params_map[p][int(np.round(v,0))]
                
        
        est = mod(**default_params, **params)
        cv = StratifiedKFold(NCV, True, seed)
        acc = np.mean(cross_val_score(est, x, y, scoring='accuracy', cv=cv, n_jobs=-1))
        return acc
    
    # run bayesopt
    start = time.time()
    BO = BayesianOptimization(mod_func, param_bounds, random_state=seed)
    BO.maximize(ninit, niter)
    print('TIME TAKEN (MIN):', (time.time() - start)/60)
    return BO

__Logistic Regression__

In [17]:
lreg_param_grid = {
    'C': np.arange(0.25,1.1,0.25),
    'penalty': ['l1', 'l2']
}

lreg_grid = grid_search(LogisticRegression(), poly_train, ytrain, lreg_param_grid, scale=MinMaxScaler())

Fitting 4 folds for each of 8 candidates, totalling 32 fits


[Parallel(n_jobs=-1)]: Done  32 out of  32 | elapsed:  1.5min finished


Best Score: 0.7273809523809524
{'C': 1.0, 'penalty': 'l1'}


In [18]:
lreg_results = lreg_grid.best_params_
lreg_results.update({'accuracy': lreg_grid.best_score_})

with open('models/lreg_results.json', 'w') as f:
    json.dump(lreg_results, f)

__LDA__

In [19]:
lda_param_grid = {
    'n_components': [None, 25, 50, 100, 150, 199]
}
lda_grid = grid_search(LinearDiscriminantAnalysis(), poly_train, ytrain, lda_param_grid)

Fitting 4 folds for each of 6 candidates, totalling 24 fits


/Users/rick.shapiro/anaconda/lib/python3.6/site-packages/numpy/core/_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)
/Users/rick.shapiro/anaconda/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/Users/rick.shapiro/anaconda/lib/python3.6/site-packages/numpy/core/_methods.py:117: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims)
/Users/rick.shapiro/anaconda/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/Users/rick.shapiro/anaconda/lib/python3.6/site-packages/numpy/core/_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)
/Users/rick.shapiro/anaconda/lib/python3.6/site-packages/numpy/core/_methods.py:32: RuntimeWarning: overflow encountered 

Best Score: 0.7117063492063492
{'n_components': None}


/Users/rick.shapiro/anaconda/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


In [20]:
lda_results = lda_grid.best_params_
lda_results.update({'accuracy': lda_grid.best_score_})

with open('models/lda_results.json', 'w') as f:
    json.dump(lda_results, f)

__KNN__

In [24]:
knn_param_grid = {
    'n_neighbors': [2**n for n in range(1,6)],
    'weights' : ['uniform', 'distance']
}

knn_grid = grid_search(KNeighborsClassifier(), poly_train, ytrain, knn_param_grid, scale=MinMaxScaler())

Fitting 4 folds for each of 10 candidates, totalling 40 fits
Best Score: 0.8353174603174603
{'n_neighbors': 2, 'weights': 'distance'}


[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  1.1min finished


In [25]:
knn_results = knn_grid.best_params_
knn_results.update({'accuracy': knn_grid.best_score_})

with open('models/knn_results.json', 'w') as f:
    json.dump(knn_results, f)

__SVM__

In [27]:
svc_param_grid = {
    'C' : [0.25, 0.5, 0.75, 1.],
    'kernel' : ['linear','rbf', 'poly', 'sigmoid']
}
svm_grid = grid_search(SVC(degree=2, max_iter=5000), poly_train, ytrain, svc_param_grid, scale=MinMaxScaler())

Fitting 4 folds for each of 16 candidates, totalling 64 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  9.0min
/Users/rick.shapiro/anaconda/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
[Parallel(n_jobs=-1)]: Done  64 out of  64 | elapsed: 15.1min finished


Best Score: 0.736574074074074
{'C': 1.0, 'kernel': 'linear'}


/Users/rick.shapiro/anaconda/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


In [29]:
svm_results = svm_grid.best_params_
svm_results.update({'accuracy': svm_grid.best_score_})

with open('models/svm_results.json', 'w') as f:
    json.dump(svm_results, f)

The next models require a lot of parameter tuning, so switching to Bayesian Optimization

__Random Forest__

In [9]:
rf_bounds = {
    'criterion': ('gini', 'entropy'),
    'n_estimators': (10,150),
    'max_depth': (3,25),
    'max_features': (2,50)
}

rf_opt = bayes_search(RandomForestClassifier, poly_train, ytrain, rf_bounds, ninit=10, niter=60)
rf_opt.res['max']

('gini', 'entropy')
{'criterion': (0, 1), 'n_estimators': (10, 150), 'max_depth': (3, 25), 'max_features': (2, 50)}
Initialization
-------------------------------------------------------------------------------------------
 Step |   Time |      Value |   criterion |   max_depth |   max_features |   n_estimators | 
    1 | 00m02s |    0.84722 |      0.0955 |     17.0612 |        14.8644 |        27.7109 | 
    2 | 00m22s |    0.85840 |      0.9250 |     17.0222 |        13.6677 |        94.9309 | 
    3 | 00m17s |    0.85099 |      0.3436 |     16.4392 |        39.8867 |        75.2569 | 
    4 | 00m05s |    0.85708 |      0.3105 |     23.4604 |        20.1433 |        43.1985 | 
    5 | 00m18s |    0.85661 |      0.0020 |     20.9630 |        40.1350 |        70.9223 | 
    6 | 00m12s |    0.85648 |      0.2356 |     19.9273 |        43.6319 |        44.1140 | 
    7 | 00m06s |    0.71243 |      0.2378 |      4.8788 |        40.1991 |        63.7376 | 
    8 | 01m33s |    0.85999 |    

/home/ec2-user/anaconda3/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([2.68977713e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 50, 'nit': 6, 'warnflag': 2}
  " state: %s" % convergence_dict)


   19 | 02m11s |    0.86085 |      1.0000 |     25.0000 |        50.0000 |       150.0000 | 
   20 | 00m14s |    0.84180 |      0.0000 |     25.0000 |        50.0000 |        10.0000 | 
   21 | 00m17s |    0.84028 |      1.0000 |     25.0000 |         2.0000 |       133.8098 | 
   22 | 00m12s |    0.84206 |      0.1904 |     24.6682 |         2.6279 |        32.9277 | 
   23 | 01m19s |    0.86131 |      0.9154 |     24.9693 |        32.7989 |       128.3471 | 


/home/ec2-user/anaconda3/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-1.16858503e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 56, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)


   24 | 00m13s |    0.83915 |      0.0000 |     25.0000 |        25.6689 |        10.0000 | 


...

/home/ec2-user/anaconda3/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:308: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


  110 | 01m31s |    0.85972 |      0.9835 |     23.8593 |        44.4629 |        61.1027 | 


/home/ec2-user/anaconda3/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:308: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


TIME TAKEN (MIN): 116.02379096349081


{'max_params': {'criterion': 0.9718043394532713,
  'max_depth': 21.21178444903386,
  'max_features': 15.674612893352394,
  'n_estimators': 118.1677200736906},
 'max_val': 0.8634259259259258}

In [14]:
rf_results = rf_opt.res['max']['max_params']
rf_results.update({'accuracy': rf_opt.res['max']['max_val']})

with open('models/rf_results.json', 'w') as f:
    json.dump(rf_results, f)

__Extra Trees__

In [15]:
et_bounds = {
    'criterion': ('gini', 'entropy'),
    'n_estimators': (10,150),
    'max_depth': (3,25),
    'max_features': (2,50)
}

et_opt = bayes_search(ExtraTreesClassifier, poly_train, ytrain, et_bounds, 
                   default_params={'bootstrap':True}, ninit=10, niter=60)
et_opt.res['max']

('gini', 'entropy')
{'criterion': (0, 1), 'n_estimators': (10, 150), 'max_depth': (3, 25), 'max_features': (2, 50)}
Initialization
-------------------------------------------------------------------------------------------
 Step |   Time |      Value |   criterion |   max_depth |   max_features |   n_estimators | 
    1 | 00m00s |    0.83915 |      0.0955 |     17.0612 |        14.8644 |        27.7109 | 
    2 | 00m02s |    0.85172 |      0.9250 |     17.0222 |        13.6677 |        94.9309 | 
    3 | 00m02s |    0.84901 |      0.3436 |     16.4392 |        39.8867 |        75.2569 | 
    4 | 00m01s |    0.85549 |      0.3105 |     23.4604 |        20.1433 |        43.1985 | 
    5 | 00m02s |    0.85899 |      0.0020 |     20.9630 |        40.1350 |        70.9223 | 
    6 | 00m01s |    0.85602 |      0.2356 |     19.9273 |        43.6319 |        44.1140 | 
    7 | 00m01s |    0.68684 |      0.2378 |      4.8788 |        40.1991 |        63.7376 | 
    8 | 00m05s |    0.85985 |    

/home/ec2-user/anaconda3/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-1.12248357e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 60, 'nit': 7, 'warnflag': 2}
  " state: %s" % convergence_dict)


   21 | 00m13s |    0.84484 |      0.0000 |     25.0000 |        50.0000 |        10.0000 | 
   22 | 00m13s |    0.83234 |      1.0000 |     25.0000 |         2.0000 |        33.1037 | 
   23 | 00m16s |    0.86032 |      1.0000 |     25.0000 |        32.8315 |       126.4220 | 
   24 | 00m12s |    0.83981 |      0.0000 |     25.0000 |        25.2145 |        10.0000 | 
   25 | 00m13s |    0.83776 |      0.0000 |     25.0000 |         2.0000 |       150.0000 | 


/home/ec2-user/anaconda3/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([1.57168897e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 50, 'nit': 6, 'warnflag': 2}
  " state: %s" % convergence_dict)


   26 | 00m14s |    0.83624 |      0.0000 |     25.0000 |         2.0000 |        93.6485 | 
   27 | 00m13s |    0.85893 |      0.4299 |     24.7896 |        49.9636 |        50.2323 | 
   28 | 00m14s |    0.85853 |      0.0703 |     24.9044 |        27.7614 |        86.8523 | 
   29 | 00m17s |    0.86118 |      0.0368 |     24.5631 |        49.6976 |       134.3498 | 


...

/home/ec2-user/anaconda3/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:308: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


   68 | 00m42s |    0.86071 |      0.9725 |     23.5680 |        41.3947 |       105.4827 | 
   69 | 00m37s |    0.85952 |      0.9921 |     23.1876 |        16.1243 |        94.3807 | 
   70 | 00m38s |    0.85390 |      1.0000 |     25.0000 |        45.3321 |        20.7050 | 


/home/ec2-user/anaconda3/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:308: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


TIME TAKEN (MIN): 23.69399822950363


{'max_params': {'criterion': 0.9566264100626243,
  'max_depth': 21.235115755441708,
  'max_features': 49.87037706868469,
  'n_estimators': 73.94741041234693},
 'max_val': 0.864021164021164}

In [16]:
et_results = et_opt.res['max']['max_params']
et_results.update({'accuracy': et_opt.res['max']['max_val']})

with open('models/et_results.json', 'w') as f:
    json.dump(et_results, f)

__MLP__

In [19]:
mlp_bounds = {
    'hidden_layer_sizes': [(25, 15, 10), (25, 25), (50,)],
    'activation' : ['logistic', 'tanh', 'relu'],
    'learning_rate_init': (0.0001, 0.1),
    'beta_1': (0.1, 0.9),
    'beta_2': (0.1, 0.999),
    'alpha' : (0.00001, 0.01)
}

mlp_default_args = {
    'random_state': seed, 
}

# the fact that this can't seem to run without spitting out an overflow warning
# makes me think I shouldn't even use it...
mlp_opt = bayes_search(MLPClassifier, poly_train, ytrain, mlp_bounds, mlp_default_args, ninit=10, niter=100)
mlp_opt.res['max']

[(25, 15, 10), (25, 25), (50,)]
['logistic', 'tanh', 'relu']
{'hidden_layer_sizes': (0, 2), 'activation': (0, 2), 'learning_rate_init': (0.0001, 0.1), 'beta_1': (0.1, 0.9), 'beta_2': (0.1, 0.999), 'alpha': (1e-05, 0.01)}
Initialization
------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   activation |     alpha |    beta_1 |    beta_2 |   hidden_layer_sizes |   learning_rate_init | 


/home/ec2-user/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)
/home/ec2-user/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)
/home/ec2-user/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)
/home/ec2-user/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)


    1 | 00m22s |    0.69226 |       0.2530 |    0.0006 |    0.3144 |    0.6667 |               0.1911 |               0.0640 | 


/home/ec2-user/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)
/home/ec2-user/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)
/home/ec2-user/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)
/home/ec2-user/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)


    2 | 00m09s |    0.59272 |       1.2133 |    0.0087 |    0.2945 |    0.1713 |               1.8500 |               0.0638 | 


...

/home/ec2-user/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)
/home/ec2-user/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)
/home/ec2-user/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)
/home/ec2-user/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)
/home/ec2-user/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/home/ec2-user/ana

  109 | 02m41s |    0.71078 |       1.5661 |    0.0100 |    0.9000 |    0.1000 |               0.0000 |               0.0001 | 


/home/ec2-user/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)
/home/ec2-user/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)
/home/ec2-user/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)
/home/ec2-user/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)
/home/ec2-user/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/home/ec2-user/ana

  110 | 03m01s |    0.71409 |       1.3882 |    0.0100 |    0.1000 |    0.1000 |               0.0000 |               0.0001 | 
TIME TAKEN (MIN): 228.9488443295161


{'max_params': {'activation': 0.7676798170835413,
  'alpha': 0.001282074070643787,
  'beta_1': 0.7366517064901443,
  'beta_2': 0.383358723763881,
  'hidden_layer_sizes': 0.47558344031675914,
  'learning_rate_init': 0.008631552538323234},
 'max_val': 0.734457671957672}

In [ ]:
# # it literally found this in the initialization...
# mlp_result={
#     'hidden_layer_sizes': 0.7677,
#     'activation': 0.0013,
#     'learning_rate_init': 0.7367,
#     'beta_1': 0.3834,
#     'beta_2': 0.4756,
#     'alpha': 0.0086,
#     'accuracy': 0.73446
# }

In [21]:
mlp_results = mlp_opt.res['max']['max_params']
mlp_results.update({'accuracy': mlp_opt.res['max']['max_val']})

with open('models/mlp_results.json', 'w') as f:
    json.dump(mlp_results, f)

__LGBM__

In [83]:
lgbm_bounds = {
    'boosting_type': ['gbdt', 'dart'],
    'max_depth': (3,25),
    'learning_rate': (0.001, 0.9),
    'n_estimators': (10,100),
    'min_split_gain': (0.001, 0.1),
    'subsample': (0.01,0.66),
    'colsample_bytree': (0.01,0.5),
    'reg_alpha': (0.00, 1.00),
    'reg_lambda': (0.00, 1.00),
    
}

lgbm_default_args = {
    'random_state': seed, 
    'objective': 'multiclass'
}


lgbm_opt = bayes_search(LGBMClassifier, poly_train, ytrain, lgbm_bounds, lgbm_default_args, ninit=10, niter=5)
lgbm_opt.res['max']

['gbdt', 'dart']
{'boosting_type': (0, 1), 'max_depth': (3, 25), 'learning_rate': (0.001, 0.9), 'n_estimators': (10, 100), 'min_split_gain': (0.001, 0.1), 'subsample': (0.01, 0.66), 'colsample_bytree': (0.01, 0.5), 'reg_alpha': (0.0, 1.0), 'reg_lambda': (0.0, 1.0)}
Initialization
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   boosting_type |   colsample_bytree |   learning_rate |   max_depth |   min_split_gain |   n_estimators |   reg_alpha |   reg_lambda |   subsample | 


/Users/rick.shapiro/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/rick.shapiro/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/rick.shapiro/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/rick.shapiro/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth valu

    1 | 00m02s |    0.68016 |          0.0955 |             0.1434 |          0.5756 |      5.7831 |           0.0634 |        34.1208 |      0.6168 |       0.0498 |      0.0464 | 


/Users/rick.shapiro/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/rick.shapiro/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/rick.shapiro/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/rick.shapiro/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth valu

    2 | 00m06s |    0.83241 |          0.9250 |             0.1238 |          0.5740 |     16.3463 |           0.0089 |        31.8769 |      0.8036 |       0.4900 |      0.5782 | 


...

/Users/rick.shapiro/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/rick.shapiro/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/rick.shapiro/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/rick.shapiro/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth valu

   14 | 00m11s |    0.59266 |          1.0000 |             0.0100 |          0.0010 |      3.0000 |           0.0010 |        60.3256 |      1.0000 |       1.0000 |      0.6600 | 


/Users/rick.shapiro/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/rick.shapiro/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/rick.shapiro/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/rick.shapiro/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth valu

   15 | 00m08s |    0.69795 |          1.0000 |             0.5000 |          0.0010 |      3.0000 |           0.0010 |        10.0000 |      1.0000 |       1.0000 |      0.6600 | 
TIME TAKEN (MIN): 2.521657117207845


{'max_params': {'boosting_type': 0.3104769418698441,
  'colsample_bytree': 0.4613561117536225,
  'learning_rate': 0.8370866784295171,
  'max_depth': 8.216906736610557,
  'min_split_gain': 0.030731358627987985,
  'n_estimators': 44.01859708278999,
  'reg_alpha': 0.38455772411904465,
  'reg_lambda': 0.37742878104895494,
  'subsample': 0.5328799468977962},
 'max_val': 0.8505291005291005}

In [85]:
lgbm_results = lgbm_opt.res['max']['max_params']
lgbm_results.update({'accuracy': lgbm_opt.res['max']['max_val']})

with open('models/lgbm_results.json', 'w') as f:
    json.dump(lgbm_results, f)

__XGBoost__

In [ ]:
xgb_bounds = {
    'booster': ['gbtree', 'dart'],
    'max_depth': (3,25),
    'learning_rate': (0.001, 0.9),
    'n_estimators': (10,100),
    'subsample': (0.01,0.66),
    'colsample_bytree': (0.01,0.5),
    'reg_alpha': (0.00, 1.00),
    'reg_lambda': (0.00, 1.00),
    'gamma': (0.00,10.00)
}

xgb_default_args = {
    'random_state': seed, 
    'silent':False,
    'objective': 'multi:softmax'
    
}


xgb_opt = bayes_search(XGBClassifier, poly_train, ytrain, xgb_bounds, xgb_default_args, ninit=10, niter=75)
xgb_opt.res['max']

In [ ]:
xgb_results = xgb_opt.res['max']['max_params']
xgb_results.update({'accuracy': xgb_opt.res['max']['max_val']})

with open('models/xgb_results.json', 'w') as f:
    json.dump(xgb_results, f)

In [31]:
# parse parameters
rf_results['criterion'] = {0:'gini', 1:'entropy'}[int(np.round(rf_results['criterion'], 0))]
rf_results['max_depth'] = int(np.round(rf_results['max_depth'], 0))
rf_results['max_features'] = int(np.round(rf_results['max_features'], 0))
rf_results['n_estimators'] = int(np.round(rf_results['n_estimators'], 0))

et_results['criterion'] = {0:'gini', 1:'entropy'}[int(np.round(et_results['criterion'], 0))]
et_results['max_depth'] = int(np.round(et_results['max_depth'], 0))
et_results['max_features'] = int(np.round(et_results['max_features'], 0))
et_results['n_estimators'] = int(np.round(et_results['n_estimators'], 0))

mlp_results['activation'] = {0:'logistic', 1:'tanh', 2:'relu'}[int(np.round(mlp_results['activation'], 0))]
mlp_results['hidden_layer_sizes'] = {0:(25, 15, 10), 1:(25, 25), 2:(50,)}[int(np.round(mlp_results['hidden_layer_sizes'], 0))]

lgbm_results['boosting_type'] = {0:'gbdt', 1:'dart'}[int(np.round(lgbm_results['boosting_type'], 0))]
lgbm_results['max_depth'] = int(np.round(lgbm_results['max_depth'], 0))
lgbm_results['n_estimators'] = int(np.round(lgbm_results['n_estimators'], 0))

xgb_results['booster'] = {0:'gbtree', 1:'dart'}[int(np.round(xgb_results['booster'], 0))]
xgb_results['max_depth'] = int(np.round(xgb_results['max_depth'], 0))
xgb_results['n_estimators'] = int(np.round(xgb_results['n_estimators'], 0))


with open('models/rf_results.json', 'w') as f:
    json.dump(rf_results, f)
with open('models/et_results.json', 'w') as f:    
    json.dump(et_results, f)
with open('models/mlp_results.json', 'w') as f:    
    json.dump(mlp_results, f)
with open('models/lgbm_results.json', 'w') as f:    
    json.dump(lgbm_results, f)
with open('models/xgb_results.json', 'w') as f:    
    json.dump(xgb_results, f)

--------------------------------------------

In [11]:
with open('models/lreg_results.json') as f:
    lreg_results = json.load(f)
with open('models/lda_results.json') as f:    
    lda_results = json.load(f)
with open('models/knn_results.json') as f:    
    knn_results = json.load(f)
with open('models/svm_results.json') as f:    
    svm_results = json.load(f)

with open('models/rf_results.json') as f:
    rf_results = json.load(f)
with open('models/et_results.json') as f:    
    et_results = json.load(f)
with open('models/mlp_results.json') as f:    
    mlp_results = json.load(f)
with open('models/lgbm_results.json') as f:    
    lgbm_results = json.load(f)
with open('models/xgb_results.json') as f:    
    xgb_results = json.load(f)

In [12]:
# all_results

all_results = {
    LogisticRegression: lreg_results,
    LinearDiscriminantAnalysis: lda_results,
    KNeighborsClassifier: knn_results,
    SVC: svm_results,
    RandomForestClassifier: rf_results,
    ExtraTreesClassifier: et_results,
    MLPClassifier: mlp_results,
    LGBMClassifier: lgbm_results,
    XGBClassifier: xgb_results
}



### Transform Test Set

In [94]:
poly_train.head()

,nb_prob_6,nb_prob_4^2,nb_prob_4 svd0 pca9,nb_prob_4^2 svd0,nb_prob_4^3,nb_prob_4^2 pca9,nb_prob_3 svd0^2,nb_prob_4 Hillshade_Noon svd0,nb_prob_6 nb_prob_4^2,nb_prob_6 nb_prob_4 svd0,nb_prob_4,nb_prob_0 nb_prob_4 svd0,nb_prob_6 svd0 Horizontal_Distance_To_Hydrology,Horizontal_Distance_To_Roadways Horizontal_Distance_To_Fire_Points pca9,nb_prob_0 Horizontal_Distance_To_Roadways pca9,nb_prob_0 pca9^2,Elevation nb_prob_4 svd0,Elevation^2 svd0,Elevation^3,nb_prob_4 pca9^2,nb_prob_4 svd1 svd0,nb_prob_4^2 svd1,nb_prob_5^3,svd1 Horizontal_Distance_To_Roadways Horizontal_Distance_To_Fire_Points,pca0 Horizontal_Distance_To_Roadways Horizontal_Distance_To_Fire_Points,nb_prob_4 pca9,nb_prob_6^2,nb_prob_3,nb_prob_6 Hillshade_9am,svd2 K3^2,nb_prob_3 svd0 PCA_K4,K3^3,nb_prob_4 Hillshade_9am pca9,nb_prob_4^2 Hillshade_9am,nb_prob_4^2 Hillshade_Noon,nb_prob_4 svd0,nb_prob_4 Hillshade_Noon pca9,nb_prob_3^2 PCA_K4,PCA_K4^3,nb_prob_3 svd0 K3,PCA_K4^2,PCA_K4,nb_prob_3 Hillshade_9am,nb_prob_3 pca0 Wilderness_Area4,nb_prob_4 Hillshade_Noon Hillshade_9am,pca0 K3^2,nb_prob_6^2 Hillshade_9am,nb_prob_6^2 nb_prob_0,nb_prob_6 nb_prob_4,nb_prob_6 Hillshade_Noon,...,nb_prob_3 svd1 svd0,nb_prob_3 pca0 svd2,Horizontal_Distance_To_Roadways pca1 svd2,Horizontal_Distance_To_Roadways pca1 Horizontal_Distance_To_Fire_Points,Horizontal_Distance_To_Roadways svd2 Horizontal_Distance_To_Fire_Points,Horizontal_Distance_To_Roadways Horizontal_Distance_To_Fire_Points,Horizontal_Distance_To_Roadways Horizontal_Distance_To_Fire_Points svd0,Horizontal_Distance_To_Roadways Horizontal_Distance_To_Fire_Points^2,Horizontal_Distance_To_Roadways Hillshade_Noon K3,Horizontal_Distance_To_Roadways Hillshade_Noon Wilderness_Area4,Horizontal_Distance_To_Roadways svd0 Hillshade_9am,Horizontal_Distance_To_Roadways svd0 Wilderness_Area4,Horizontal_Distance_To_Roadways svd0 K4,Horizontal_Distance_To_Roadways svd0 svd3,Horizontal_Distance_To_Roadways svd5 Wilderness_Area4,Horizontal_Distance_To_Roadways Hillshade_3pm K3,Horizontal_Distance_To_Roadways Hillshade_3pm Wilderness_Area4,Horizontal_Distance_To_Roadways K3 svd6,Horizontal_Distance_To_Roadways K3 pca6,Horizontal_Distance_To_Roadways Wilderness_Area4 svd6,Horizontal_Distance_To_Roadways Wilderness_Area4 Aspect,Horizontal_Distance_To_Roadways Wilderness_Area4 pca3,Horizontal_Distance_To_Roadways Wilderness_Area4 pca6,Horizontal_Distance_To_Roadways^2 pca1,Horizontal_Distance_To_Roadways^2 svd2,Horizontal_Distance_To_Roadways^2 svd0,Horizontal_Distance_To_Roadways^2 K3,Horizontal_Distance_To_Roadways^2 Wilderness_Area4,pca1 Horizontal_Distance_To_Fire_Points Horizontal_Distance_To_Hydrology,pca1 Horizontal_Distance_To_Fire_Points^2,pca1 svd0 pca9,pca1 svd0 K4,pca1^3,svd2 Horizontal_Distance_To_Fire_Points^2,svd2 svd0 pca9,svd2 svd0 K4,svd2 Hillshade_3pm Vertical_Distance_To_Hydrology,svd2^2 Horizontal_Distance_To_Fire_Points,svd2^3,Horizontal_Distance_To_Fire_Points svd0 svd5,Horizontal_Distance_To_Fire_Points svd0 pca5,Horizontal_Distance_To_Fire_Points svd0 svd6,Horizontal_Distance_To_Fire_Points svd0 pca6,Horizontal_Distance_To_Fire_Points Hillshade_9am K3,Horizontal_Distance_To_Fire_Points Hillshade_3pm pca9,Horizontal_Distance_To_Fire_Points Vertical_Distance_To_Hydrology K3,Horizontal_Distance_To_Fire_Points Vertical_Distance_To_Hydrology Wilderness_Area4,Horizontal_Distance_To_Fire_Points Vertical_Distance_To_Hydrology^2,Horizontal_Distance_To_Fire_Points pca9 K3,Horizontal_Distance_To_Fire_Points K3
0,0.01,0.00,-0.00,0.01,0.00,-0.00,0.00,0.08,0.00,0.00,0.06,0.02,0.00,-0.00,-0.00,0.00,0.03,0.21,0.05,0.00,-0.01,-0.00,0.00,-0.01,-0.01,-0.00,0.00,0.00,0.01,4.80,0.00,8.00,-0.00,0.00,0.00,0.09,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,0.04,-0.32,0.00,0.00,0.00,0.01,...,-0.00,-0.00,0.11,0.08,0.08,0.07,0.10,0.06,0.13,0.00,0.10,0.00,0.12,0.00,-0.00,0.09,0.00,-0.05,-0.02,-0.00,0.00,0.00,-0.00,0.01,0.01,0.01,0.01,0.00,0.21,0.96,-0.05,1.88,1.71,0.97,-0.05,1.89,0.15,1.29,1.73,-0.81,-0.43,-0.46,-0.22,1.56,-0.01,0.37,0.00,0.04,-0.05,1.80
1

In [109]:
def parse_column(col):
    cols = col.split(' ')
    exp_cols = [c for c in cols if '^' in c]
    interaction_terms = [c for c in cols if c not in exp_cols]
    exp_dict = dict({c.split("^")[0]:int(c.split("^")[1])  for c in exp_cols})
    return exp_dict, interaction_terms

def create_ftr(df, exp_terms, inter_terms):
    return pd.concat([df[col].pow(ex) for col, ex in exp_terms.items()] + [df[i] for i in inter_terms], axis=1)\
             .prod(1)

    

In [115]:
all_test_df.shape

(565892, 115)

In [116]:
poly_test_cols = []
for col in poly_train.columns:
    if col in all_test_df.columns:
        poly_test_cols.append(all_test_df[col])
        continue
    else:
        exp_terms, inter_terms = parse_column(col)
        poly_test_cols.append(create_ftr(all_test_df, exp_terms, inter_terms))

poly_test = pd.concat(poly_test_cols, axis=1)
poly_test.shape

(565892, 200)

In [119]:
poly_test.columns = poly_train.columns

In [120]:
poly_test.to_csv('clean_data/test_poly_final.csv',index=False)

In [121]:
poly_test.head()

,nb_prob_6,nb_prob_4^2,nb_prob_4 svd0 pca9,nb_prob_4^2 svd0,nb_prob_4^3,nb_prob_4^2 pca9,nb_prob_3 svd0^2,nb_prob_4 Hillshade_Noon svd0,nb_prob_6 nb_prob_4^2,nb_prob_6 nb_prob_4 svd0,nb_prob_4,nb_prob_0 nb_prob_4 svd0,nb_prob_6 svd0 Horizontal_Distance_To_Hydrology,Horizontal_Distance_To_Roadways Horizontal_Distance_To_Fire_Points pca9,nb_prob_0 Horizontal_Distance_To_Roadways pca9,nb_prob_0 pca9^2,Elevation nb_prob_4 svd0,Elevation^2 svd0,Elevation^3,nb_prob_4 pca9^2,nb_prob_4 svd1 svd0,nb_prob_4^2 svd1,nb_prob_5^3,svd1 Horizontal_Distance_To_Roadways Horizontal_Distance_To_Fire_Points,pca0 Horizontal_Distance_To_Roadways Horizontal_Distance_To_Fire_Points,nb_prob_4 pca9,nb_prob_6^2,nb_prob_3,nb_prob_6 Hillshade_9am,svd2 K3^2,nb_prob_3 svd0 PCA_K4,K3^3,nb_prob_4 Hillshade_9am pca9,nb_prob_4^2 Hillshade_9am,nb_prob_4^2 Hillshade_Noon,nb_prob_4 svd0,nb_prob_4 Hillshade_Noon pca9,nb_prob_3^2 PCA_K4,PCA_K4^3,nb_prob_3 svd0 K3,PCA_K4^2,PCA_K4,nb_prob_3 Hillshade_9am,nb_prob_3 pca0 Wilderness_Area4,nb_prob_4 Hillshade_Noon Hillshade_9am,pca0 K3^2,nb_prob_6^2 Hillshade_9am,nb_prob_6^2 nb_prob_0,nb_prob_6 nb_prob_4,nb_prob_6 Hillshade_Noon,...,nb_prob_3 svd1 svd0,nb_prob_3 pca0 svd2,Horizontal_Distance_To_Roadways pca1 svd2,Horizontal_Distance_To_Roadways pca1 Horizontal_Distance_To_Fire_Points,Horizontal_Distance_To_Roadways svd2 Horizontal_Distance_To_Fire_Points,Horizontal_Distance_To_Roadways Horizontal_Distance_To_Fire_Points,Horizontal_Distance_To_Roadways Horizontal_Distance_To_Fire_Points svd0,Horizontal_Distance_To_Roadways Horizontal_Distance_To_Fire_Points^2,Horizontal_Distance_To_Roadways Hillshade_Noon K3,Horizontal_Distance_To_Roadways Hillshade_Noon Wilderness_Area4,Horizontal_Distance_To_Roadways svd0 Hillshade_9am,Horizontal_Distance_To_Roadways svd0 Wilderness_Area4,Horizontal_Distance_To_Roadways svd0 K4,Horizontal_Distance_To_Roadways svd0 svd3,Horizontal_Distance_To_Roadways svd5 Wilderness_Area4,Horizontal_Distance_To_Roadways Hillshade_3pm K3,Horizontal_Distance_To_Roadways Hillshade_3pm Wilderness_Area4,Horizontal_Distance_To_Roadways K3 svd6,Horizontal_Distance_To_Roadways K3 pca6,Horizontal_Distance_To_Roadways Wilderness_Area4 svd6,Horizontal_Distance_To_Roadways Wilderness_Area4 Aspect,Horizontal_Distance_To_Roadways Wilderness_Area4 pca3,Horizontal_Distance_To_Roadways Wilderness_Area4 pca6,Horizontal_Distance_To_Roadways^2 pca1,Horizontal_Distance_To_Roadways^2 svd2,Horizontal_Distance_To_Roadways^2 svd0,Horizontal_Distance_To_Roadways^2 K3,Horizontal_Distance_To_Roadways^2 Wilderness_Area4,pca1 Horizontal_Distance_To_Fire_Points Horizontal_Distance_To_Hydrology,pca1 Horizontal_Distance_To_Fire_Points^2,pca1 svd0 pca9,pca1 svd0 K4,pca1^3,svd2 Horizontal_Distance_To_Fire_Points^2,svd2 svd0 pca9,svd2 svd0 K4,svd2 Hillshade_3pm Vertical_Distance_To_Hydrology,svd2^2 Horizontal_Distance_To_Fire_Points,svd2^3,Horizontal_Distance_To_Fire_Points svd0 svd5,Horizontal_Distance_To_Fire_Points svd0 pca5,Horizontal_Distance_To_Fire_Points svd0 svd6,Horizontal_Distance_To_Fire_Points svd0 pca6,Horizontal_Distance_To_Fire_Points Hillshade_9am K3,Horizontal_Distance_To_Fire_Points Hillshade_3pm pca9,Horizontal_Distance_To_Fire_Points Vertical_Distance_To_Hydrology K3,Horizontal_Distance_To_Fire_Points Vertical_Distance_To_Hydrology Wilderness_Area4,Horizontal_Distance_To_Fire_Points Vertical_Distance_To_Hydrology^2,Horizontal_Distance_To_Fire_Points pca9 K3,Horizontal_Distance_To_Fire_Points K3
0,0.01,0.00,-0.01,0.01,0.00,-0.00,0.00,0.08,0.00,0.00,0.06,0.02,0.00,-0.03,-0.00,0.00,0.04,0.30,0.07,0.00,-0.00,-0.00,0.00,-0.01,-0.03,-0.00,0.00,0.00,0.01,4.67,0.00,8,-0.00,0.00,0.00,0.10,-0.00,0.00,0,0.00,0,0,0.00,-0.00,0.03,-0.34,0.00,0.00,0.00,0.01,...,-0.00,-0.00,0.52,0.43,0.43,0.37,0.67,0.35,0.58,0.00,0.54,0.00,0.70,0.32,-0.00,0.49,0.00,-0.12,0.17,-0.00,0.00,-0.00,0.00,0.17,0.18,0.27,0.30,0.00,0.00,1.04,-0.15,2.07,1.51,1.05,-0.15,2.10,0.15,1.30,1.59,-1.05,-0.23,-0.25,0.38,1.47,-0.04,0.40,0.00,0.04,-0.13,1.90
1,0.01,0.00,-0.

### Submit Predictions

In [13]:
test_id = test.Id
xtest = poly_test

In [14]:
def write_submission(mod, params, xtrain, ytrain, test_id, xtest, scale = None):

    if mod == SVC:
        est = mod(**params, probability=True)
    else:
        est = mod(**params)
        
    if scale:
        xtrain = scale.fit_transform(xtrain)
        xtest = scale.transform(xtest)
    est.fit(xtrain,ytrain)
    probs = est.predict_proba(xtest)
    preds = np.argmax(probs, 1)+1
    
    
    name = repr(est.__class__).split('.')[-1].strip('>').strip("'")
    est_id = len([s for s in os.listdir('Submissions/') if s.startswith(name)])+1
    fname = "{}{}_{}".format(name, est_id, dt.date.today().strftime('%Y%m%d'))
    
    probs_df = pd.concat([pd.Series(test_id), pd.DataFrame(probs)], axis=1)
    probs_df.columns = ['Id'] + [name + '_CLASS_'+str(i) for i in range(ytrain.nunique())]
    probs_df.to_csv('Submissions/PROBS_'+fname+'.csv', index=False)
    
    preds_df = pd.concat([pd.Series(test_id), pd.Series(preds)], axis=1)
    preds_df.columns = ['Id', 'Cover_Type']
    preds_df.to_csv('Submissions/{}.csv'.format(fname), index=False)
    
    # write model params
    with open('Submissions/{}_params.json'.format(fname), 'w') as f:
        json.dump(params, f)
        
    return 
    
    

In [151]:
for mod, params in all_results.items():
    print(mod)
    write_submission(mod, {k:v for k,v in params.items() if k!='accuracy'}, poly_train, ytrain, test_id, xtest, MinMaxScaler())

<class 'sklearn.linear_model.logistic.LogisticRegression'>
<class 'sklearn.discriminant_analysis.LinearDiscriminantAnalysis'>


/Users/rick.shapiro/anaconda/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/Users/rick.shapiro/anaconda/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


<class 'sklearn.neighbors.classification.KNeighborsClassifier'>
<class 'sklearn.svm.classes.SVC'>
<class 'sklearn.ensemble.forest.RandomForestClassifier'>
<class 'sklearn.ensemble.forest.ExtraTreesClassifier'>
<class 'sklearn.neural_network.multilayer_perceptron.MLPClassifier'>
<class 'lightgbm.sklearn.LGBMClassifier'>
<class 'xgboost.sklearn.XGBClassifier'>


### Ensembling!

__Majority Vote__

First thing I'm going to try is just combine a few of the predictions I already made. 
Best Practice would cross-validate these combinations on the training set, but I'm just going to submit a few manually.

The Majority Vote can just me the mode of the preds, or the max of the (optionally weighted) average of their probabilities.

__Stacking__

Next I'm going to use the fitted estimators to to generate training_predictions, and then build meta-estimators
on those predictions. Then use the meta-estimators to generate preds for the test set.


In [9]:
poly_test = pd.read_csv('clean_data/test_poly_final.csv')

In [10]:
poly_test = reduce_mem_usage(poly_test)
poly_test.shape

Memory usage of dataframe is 863.48 MB
Memory usage after optimization is: 212.09 MB
Decreased by 75.4%


(565892, 200)

_Load In Submission Files_

In [152]:
preds = ['Submissions/' + f for f in os.listdir('Submissions/') if (not f.startswith('PROBS')) and (f.endswith('.csv'))]
probs = ['Submissions/' + f for f in os.listdir('Submissions/') if (f.startswith('PROBS')) and (f.endswith('.csv'))]
preds, probs

(['Submissions/ExtraTreesClassifier1_20180921.csv',
  'Submissions/KNeighborsClassifier1_20180921.csv',
  'Submissions/LGBMClassifier1_20180921.csv',
  'Submissions/LinearDiscriminantAnalysis1_20180921.csv',
  'Submissions/LogisticRegression1_20180921.csv',
  'Submissions/MLPClassifier1_20180921.csv',
  'Submissions/pred_mode_ExtraTreesClassifier1_KNeighborsClassifier1_RandomForestClassifier1_XGBClassifier1_LGBMClassifier1_20180921.csv',
  'Submissions/prob_mean_ExtraTreesClassifier1_KNeighborsClassifier1_RandomForestClassifier1_XGBClassifier1_20180921.csv',
  'Submissions/prob_mean_ExtraTreesClassifier1_KNeighborsClassifier1_RandomForestClassifier1_XGBClassifier1_LGBMClassifier1_20180921.csv',
  'Submissions/prob_mean_ExtraTreesClassifier1_RandomForestClassifier1_XGBClassifier1_LGBMClassifier1_20180921.csv',
  'Submissions/RandomForestClassifier1_20180921.csv',
  'Submissions/SVC1_20180921.csv',
  'Submissions/XGBClassifier1_20180921.csv'],
 ['Submissions/PROBS_ExtraTreesClassifier1_2

In [153]:
preds_dict = dict(zip(
    [fp.split('_')[0].split('/')[-1] for fp in preds],
    [pd.read_csv(fp) for fp in preds]
))

prob_dict = dict(zip(
    [fp.split('_')[1] for fp in probs],
    [pd.read_csv(fp) for fp in probs]
))

In [154]:
[p.shape for p in preds_dict.values()], [p.shape for p in prob_dict.values()]

([(565892, 2),
  (565892, 2),
  (565892, 2),
  (565892, 2),
  (565892, 2),
  (565892, 2),
  (565892, 2),
  (565892, 2),
  (565892, 2),
  (565892, 2),
  (565892, 2)],
 [(565892, 8),
  (565892, 8),
  (565892, 8),
  (565892, 8),
  (565892, 8),
  (565892, 8),
  (565892, 8),
  (565892, 8),
  (565892, 8)])

In [155]:
# from itertools import product, combinations
# combos = [list(combinations(preds_dict.keys(), n)) for n in range(2,len(preds_dict)+1)]
# combos = [c for inner in combos for c in inner]
# len(combos)

### Voting

In [156]:
def majority_vote(preds):
    'Preds= Dict of Name: Dataframe. DF has cols ["Id", "Cover_Type"]'
    df = pd.DataFrame(columns=['Id'])
    for k,v in preds.items():
        df = df.merge(v.rename(columns={'Cover_Type':'Cover_Type_'+k}), 'outer', 'Id')
    modes = np.ravel(stats.mode(df.set_index('Id').values, 1)[0])
    return pd.DataFrame({'Id': df.Id.values, 'Cover_Type': modes})

def prob_mean(probs):
    "probs= List Of DataFrames"
    n_dfs = len(probs)
    sum_probs = probs[0].set_index('Id').values.copy()
    for df in probs[1:]:
        sum_probs += df.set_index('Id').values
    mean_probs = sum_probs/n_dfs
    
    return pd.DataFrame({'Id': probs[0].Id, 'Cover_Type': np.argmax(mean_probs,1)+1})


def write_ensemble_submission(df, mod_names, ensemble_type = 'pred_mode'): # pred_mode, 'prob_mean', 'weighted_avg'
    fname = ensemble_type + '_' + '_'.join(mod_names) + '_' + dt.date.today().strftime('%Y%m%d') + '.csv'
    df.to_csv('Submissions/'+fname, index=False)

In [157]:
prob_dict.keys()

dict_keys(['ExtraTreesClassifier1', 'KNeighborsClassifier1', 'LGBMClassifier1', 'LinearDiscriminantAnalysis1', 'LogisticRegression1', 'MLPClassifier1', 'RandomForestClassifier1', 'SVC1', 'XGBClassifier1'])

In [158]:
maj_vote = majority_vote({k:v for k,v in preds_dict.items() if k in 
                          ['ExtraTreesClassifier1', 'KNeighborsClassifier1', 'RandomForestClassifier1', 'XGBClassifier1', 'LGBMClassifier1']})

write_ensemble_submission(maj_vote, ['ExtraTreesClassifier1', 'KNeighborsClassifier1', 'RandomForestClassifier1', 'XGBClassifier1', 'LGBMClassifier1'])

In [159]:
prob_preds = prob_mean([v for k,v in prob_dict.items() if k in 
                          ['ExtraTreesClassifier1', 'KNeighborsClassifier1', 'RandomForestClassifier1', 'XGBClassifier1', 'LGBMClassifier1']])
write_ensemble_submission(prob_preds,
                          ['ExtraTreesClassifier1', 'KNeighborsClassifier1', 'RandomForestClassifier1', 'XGBClassifier1', 'LGBMClassifier1'],
                          'prob_mean')